In [ ]:
import os

import cv2
import numpy as np


def print_comparison(image, dct_coeffs, reconstructed_image):
    dct_orig = np.log(np.abs(cv2.dct(np.float32(image))) + 1)
    dct_mod = np.log(np.abs(dct_coeffs) + 1)

    plt.figure(figsize=(16, 8))
    plt.subplot(2, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    
    plt.subplot(2, 2, 2)
    plt.imshow(dct_orig, cmap='hot')
    plt.title('DCT original')
    plt.colorbar()
    
    
    plt.subplot(2, 2, 3)
    plt.imshow(reconstructed_image, cmap='gray')
    plt.title('Reconstructed Image')
    
    plt.subplot(2, 2, 4)
    plt.imshow(dct_mod, cmap='hot')
    plt.title('DCT GAN fingerprint')
    plt.colorbar()
    
    plt.show()

## Frequency Bars Attack

In [ ]:
image = cv2.imread('../../data/PLUS/train/genuine/001-PLUS-FV3-Laser_PALMAR_001_01_02_01.png', cv2.IMREAD_GRAYSCALE)


###### Start ATTACK ######
# Apply 2D DCT
dct_result = cv2.dct(np.float32(image))
#set the last s rows and columns to zero
s=60
dct_result[-s:,:] = 0
dct_result[:,-s:] = 0
reconstructed_image = cv2.idct(dct_result)
###### End ATTACK ######

print_comparison(image, dct_result, reconstructed_image)

## Mean Spectrum Attack

In [ ]:
# read filenames in a specified directory
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

filenames_genuine = glob.glob('../../data/PLUS/train/genuine/*.png')
filenames_cycle003 = glob.glob('../../data/PLUS/train/spoofed_synthethic_cyclegan_003/*.png')

#create mean of dct coeffiecents of filenames_genuine
mean_dct_genuine = np.zeros(cv2.imread(filenames_genuine[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_genuine:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_genuine += cv2.dct(np.float32(image))
    
mean_dct_genuine /= len(filenames_genuine)

#create mean of dct coeffiecents of filenames_cycle003
mean_dct_cycle003 = np.zeros(cv2.imread(filenames_cycle003[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_cycle003:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_cycle003 += cv2.dct(np.float32(image))
    
mean_dct_cycle003 /= len(filenames_cycle003)

# Needs to be repeated for each gan class and applied on corresponding val images!!!!
F_m = mean_dct_cycle003 - mean_dct_genuine


###### Start ATTACK ######
s=0.5
filenames_cycle003_val = glob.glob('../../data/PLUS/val/spoofed_synthethic_cyclegan_003/*.png')
for filename in filenames_cycle003_val[:1]:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    dct_result = cv2.dct(np.float32(image))
    modified_coeffs = dct_result - (s*F_m)
    #store the new image in removal folder
    reconstructed_image = cv2.idct(modified_coeffs)
    print_comparison(image, F_m, reconstructed_image)
###### End ATTACK ######


## Frequency Peaks Attack

In [ ]:
# read filenames in a specified directory
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

filenames_genuine = glob.glob('../../data/PLUS/train/genuine/*.png')
filenames_cycle003 = glob.glob('../../data/PLUS/train/spoofed_synthethic_drit_004/*.png')

#create mean of dct coeffiecents of filenames_genuine
mean_dct_genuine = np.zeros(cv2.imread(filenames_genuine[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_genuine:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_genuine += np.log(np.abs(cv2.dct(np.float32(image))))
    
mean_dct_genuine /= len(filenames_genuine)

#create mean of dct coeffiecents of filenames_cycle003
mean_dct_cycle003 = np.zeros(cv2.imread(filenames_cycle003[0], cv2.IMREAD_GRAYSCALE).shape)
for filename in filenames_cycle003:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    mean_dct_cycle003 += np.log(np.abs(cv2.dct(np.float32(image))))
    
mean_dct_cycle003 /= len(filenames_cycle003)

# Needs to be repeated for each gan class and applied on corresponding val images!!!!
F_p = np.exp(mean_dct_cycle003 - mean_dct_genuine)

###### Start ATTACK ######
# scale F_p to [0,1]
F_p = (F_p - np.min(F_p)) / (np.max(F_p) - np.min(F_p))
t=0.1
#set values under threshold t to zero
F_p[F_p < t] = 0
s=100
F_p *= s
#scale F_p to [0,1]
F_p = (F_p - np.min(F_p)) / (np.max(F_p) - np.min(F_p))

filenames_cycle003_val = glob.glob('../../data/PLUS/val/spoofed_synthethic_drit_004/*.png')
for filename in filenames_cycle003_val[:1]:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    dct_result = cv2.dct(np.float32(image))
    modified_coeffs = dct_result * (1-F_p)
    #store the new image in removal folder
    reconstructed_image = cv2.idct(modified_coeffs)
    print_comparison(image, F_p, reconstructed_image)
###### End ATTACK ######

## Regression Weights Attack

In [ ]:
# train a lasso model on genuine dct coeffs
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso

filenames_genuine = glob.glob('../../data/PLUS/train/genuine/*.png')
filenames_cycle003 = glob.glob('../../data/PLUS/train/spoofed_synthethic_cyclegan_003/*.png')

#create mean of dct coeffiecents of filenames_genuine
lasso_model = Lasso(alpha=0.1) 
for filename in filenames_genuine:
    for filename_cycle003 in filenames_cycle003:
        image_gen = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        image_gan = cv2.imread(filename_cycle003, cv2.IMREAD_GRAYSCALE)
        dct_result_gen = cv2.dct(np.float32(image_gen))
        dct_result_gan = cv2.dct(np.float32(image_gan))
        lasso_model.fit(dct_result_gan, dct_result_gen)

###### Start ATTACK ######        
F_r = lasso_model.coef_
# scale F_r to [-1,1]
F_r = 2 * (F_r - np.min(F_r)) / (np.max(F_r) - np.min(F_r)) - 1
filenames_cycle003_val = glob.glob('../../data/PLUS/val/spoofed_synthethic_cyclegan_003/*.png')
for filename in filenames_cycle003_val[:1]:
    image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    dct_result = cv2.dct(np.float32(image))
    
    modified_coeffs = dct_result * (1-F_r)
    #store the new image in removal folder
    reconstructed_image = cv2.idct(modified_coeffs)
    print_comparison(image, F_r, reconstructed_image)
###### End ATTACK ######
